In [84]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rcParams['figure.dpi'] = 150

import astropy
from astropy.table import Table, join, vstack
from astropy import units as u
from astropy.coordinates import SkyCoord

Read in data:

In [3]:
tab_quaia = Table.read('../data/quaia_G20.5.fits')
print("Columns:", tab_quaia.columns)
print("N =", len(tab_quaia))

Columns: <TableColumns names=('source_id','unwise_objid','redshift_quaia','redshift_quaia_err','ra','dec','l','b','phot_g_mean_mag','phot_bp_mean_mag','phot_rp_mean_mag','mag_w1_vg','mag_w2_vg','pm','pmra','pmdec','pmra_error','pmdec_error')>
N = 1295502


In [9]:
tab_sdss = Table.read('../data/sdss_specall.csv')
print("Columns:", tab_sdss.columns)
print("N =", len(tab_sdss))

Columns: <TableColumns names=('specObjID','ra','dec','class','subClass','z','zWarning','sciencePrimary')>
N = 5112724


In [10]:
set(tab_sdss['class'])

{'GALAXY', 'QSO', 'STAR'}

These should all be "sciencePrimary" because we got them from the specObj view of the table in SkyServer (not SpecObjAll)

In [23]:
set(tab_sdss['sciencePrimary'])

{1}

In [43]:
tab_sdss_dr16q = Table.read('../data/SDSS_DR16Q_v4.fits')
tab_sdss_dr16q = tab_sdss_dr16q[tab_sdss_dr16q['OBJID'].mask==False] #remove ones with no objid (1001)
print("Columns:", tab_sdss_dr16q.columns)
print("N =", len(tab_sdss_dr16q))

Columns: <TableColumns names=('SDSS_NAME','RA','DEC','PLATE','MJD','FIBERID','AUTOCLASS_PQN','AUTOCLASS_DR14Q','IS_QSO_QN','Z_QN','RANDOM_SELECT','Z_10K','Z_CONF_10K','PIPE_CORR_10K','IS_QSO_10K','THING_ID','Z_VI','Z_CONF','CLASS_PERSON','Z_DR12Q','IS_QSO_DR12Q','Z_DR7Q_SCH','IS_QSO_DR7Q','Z_DR6Q_HW','Z_DR7Q_HW','IS_QSO_FINAL','Z','SOURCE_Z','Z_PIPE','ZWARNING','OBJID','Z_PCA','ZWARN_PCA','DELTACHI2_PCA','Z_HALPHA','ZWARN_HALPHA','DELTACHI2_HALPHA','Z_HBETA','ZWARN_HBETA','DELTACHI2_HBETA','Z_MGII','ZWARN_MGII','DELTACHI2_MGII','Z_CIII','ZWARN_CIII','DELTACHI2_CIII','Z_CIV','ZWARN_CIV','DELTACHI2_CIV','Z_LYA','ZWARN_LYA','DELTACHI2_LYA','Z_LYAWG','Z_DLA','NHI_DLA','CONF_DLA','BAL_PROB','BI_CIV','ERR_BI_CIV','AI_CIV','ERR_AI_CIV','BI_SIIV','ERR_BI_SIIV','AI_SIIV','ERR_AI_SIIV','BOSS_TARGET1','EBOSS_TARGET0','EBOSS_TARGET1','EBOSS_TARGET2','ANCILLARY_TARGET1','ANCILLARY_TARGET2','NSPEC_SDSS','NSPEC_BOSS','NSPEC','PLATE_DUPLICATE','MJD_DUPLICATE','FIBERID_DUPLICATE','SPECTRO_DUPLICATE','S

In [44]:
tab_sdss_q = tab_sdss[tab_sdss['class']=='QSO']
print(len(tab_sdss_q))

1105286


Do cross-matching:

In [18]:
def cross_match(ra1, dec1, ra2, dec2, separation):
    coords1 = SkyCoord(ra=ra1, dec=dec1, frame='icrs')    
    coords2 = SkyCoord(ra=ra2, dec=dec2, frame='icrs') 
    cross = astropy.coordinates.search_around_sky(coords1, coords2, separation) 
    index_list_1in2, index_list_2in1 = cross[0], cross[1] 
    return index_list_1in2, index_list_2in1

In [19]:
# Perform cross-match; 1 arcsec is a reasonable separation
separation = 1*u.arcsec
index_list_quaiaINsdss, index_list_sdssINquaia = cross_match(tab_quaia['ra'], tab_quaia['dec'],
                                                           tab_sdss['ra']*u.degree, tab_sdss['dec']*u.degree,
                                                           separation=separation)

In [25]:
tab_quaia_with_sdss_match = tab_quaia[index_list_quaiaINsdss]
tab_sdss_with_quaia_match = tab_sdss[index_list_sdssINquaia]

# Can add info from SDSS table to Quaia table, as these should have the same length now, e.g.:
for column in tab_sdss.columns:
    tab_quaia_with_sdss_match.add_column(tab_sdss_with_quaia_match[column], name='sdss_'+column)

# Print number in each table; last two lines should be identical
print(f'Number of Quaia quasars: {len(tab_quaia)}')
print(f'Number of SDSS sources: {len(tab_sdss)}')
print(f'Number of Quaia quasars with SDSS match: {len(tab_quaia_with_sdss_match)}')
print(f'Number of SDSS sources with Gaia match: {len(tab_sdss_with_quaia_match)}')

Number of Quaia quasars: 1295502
Number of SDSS sources: 5112724
Number of Quaia quasars with SDSS match: 303203
Number of SDSS sources with Gaia match: 303203


In [27]:
tab_quaia_with_sdss_match.columns

<TableColumns names=('source_id','unwise_objid','redshift_quaia','redshift_quaia_err','ra','dec','l','b','phot_g_mean_mag','phot_bp_mean_mag','phot_rp_mean_mag','mag_w1_vg','mag_w2_vg','pm','pmra','pmdec','pmra_error','pmdec_error','sdss_specObjID','sdss_ra','sdss_dec','sdss_class','sdss_subClass','sdss_z','sdss_zWarning','sdss_sciencePrimary')>

In [28]:
tab_quaia_with_sdss_match_nozw = tab_quaia_with_sdss_match[tab_quaia_with_sdss_match['sdss_zWarning']==0]
print(f'Number of Quaia quasars with SDSS match with no z-warning: {len(tab_quaia_with_sdss_match_nozw)}')

Number of Quaia quasars with SDSS match with no z-warning: 294693


Note: for the galaxy sample for quaia, we excluded galaxies with subClass='AGN' or 'AGN BROADLINE'; we have not done that here

In [39]:
print(f"Number of matched stars: {np.sum(tab_quaia_with_sdss_match['sdss_class']=='STAR')}")
print(f"Number of matched stars with no z-warning: {np.sum(tab_quaia_with_sdss_match_nozw['sdss_class']=='STAR')}")
print()
print(f"Number of matched galaxies: {np.sum(tab_quaia_with_sdss_match['sdss_class']=='GALAXY')}")
print(f"Number of matched galaxies with no z-warning: {np.sum(tab_quaia_with_sdss_match_nozw['sdss_class']=='GALAXY')}")
print()
print(f"Number of matched QSOs: {np.sum(tab_quaia_with_sdss_match['sdss_class']=='QSO')}")
print(f"Number of matched QSOs with no z-warning: {np.sum(tab_quaia_with_sdss_match_nozw['sdss_class']=='QSO')}")

Number of matched stars: 273
Number of matched stars with no z-warning: 123

Number of matched galaxies: 2124
Number of matched galaxies with no z-warning: 669

Number of matched QSOs: 300806
Number of matched QSOs with no z-warning: 293901


Now let's check the downloaded SDSS DR16 catalog:

In [52]:
# Perform cross-match; 1 arcsec is a reasonable separation
separation = 1*u.arcsec
index_list_quaiaINsdssdr16q, index_list_sdssdr16qINquaia = cross_match(tab_quaia['ra'], tab_quaia['dec'],
                                                           tab_sdss_dr16q['RA']*u.degree, tab_sdss_dr16q['DEC']*u.degree,
                                                           separation=separation)

In [81]:
tab_quaia_with_sdssdr16q_match = tab_quaia[index_list_quaiaINsdssdr16q]
tab_sdssdr16q_with_quaia_match = tab_sdss_dr16q[index_list_sdssdr16qINquaia]

# Can add info from SDSS table to Quaia table, as these should have the same length now, e.g.:
for column in tab_sdss_dr16q.columns:
    tab_quaia_with_sdssdr16q_match.add_column(tab_sdssdr16q_with_quaia_match[column], name='sdss_'+column)

# Print number in each table; last two lines should be identical
print(f'Number of Quaia quasars with SDSS  DR16Q match: {len(tab_quaia_with_sdssdr16q_match)}')
print(f'Number of SDSS DR16Q sources with Gaia match: {len(tab_sdssdr16q_with_quaia_match)}')

Number of Quaia quasars with SDSS  DR16Q match: 294149
Number of SDSS DR16Q sources with Gaia match: 294149


So we got a few more here than with our SkyServer download; let's check em out

In [59]:
tab_quaia_with_sdss_match.add_index('source_id')
tab_quaia_with_sdssdr16q_match.add_index('source_id')

In [76]:
tab_quaia_sdss_compare.add_index('source_id')

In [83]:
print("All for those with Quaia matches:")
# output isin: Has the same shape as element. The values element[isin] are in test_elements.
i_sdss_in_sdssdr16q = np.isin(tab_quaia_with_sdss_match['source_id'], tab_quaia_with_sdssdr16q_match['source_id'])
print('Number of SDSS QSOs in SDSS DR16Q:', np.sum(i_sdss_in_sdssdr16q))
print('Number of SDSS QSOs NOT in SDSS DR16Q:', np.sum(~i_sdss_in_sdssdr16q))

i_sdssdr16q_in_sdss = np.isin(tab_quaia_with_sdssdr16q_match['source_id'], tab_quaia_with_sdss_match['source_id'])
print('Number of SDSS DR16Q sources in SDSS QSOs:', np.sum(i_sdssdr16q_in_sdss))
print('Number of SDSS DR16Q sources NOT in SDSS QSOs:', np.sum(~i_sdssdr16q_in_sdss))

All for those with Quaia matches:
Number of SDSS QSOs in SDSS DR16Q: 294060
Number of SDSS QSOs NOT in SDSS DR16Q: 9143
Number of SDSS DR16Q sources in SDSS QSOs: 294062
Number of SDSS DR16Q sources NOT in SDSS QSOs: 87


So here's our final table of Quaia souces with SDSS spectra:

In [90]:
tab_quaia_sdss_combined = join(tab_quaia_with_sdss_match, tab_quaia_with_sdssdr16q_match, keys='source_id',
                              join_type='outer')
print(f'Number of Quaia quasars with any SDSS spectrum: {len(tab_quaia_sdss_combined)}')

Number of Quaia quasars with any SDSS spectrum: 303292


In [89]:
tab_quaia_sdss_combined_nozw = tab_quaia_sdss_combined[tab_quaia_sdss_combined['sdss_zWarning']==0]
print(f'Number of Quaia quasars with any SDSS spectrum with no z-warning: {len(tab_quaia_sdss_combined_nozw)}')

Number of Quaia quasars with any SDSS spectrum with no z-warning: 294695
